# Task 9 (Topic Modelling)

## Scraping Data (Youtube)

### Crawling data From Youtube

**Tujuan** dari program ini adalah melakukan crawling (pengambilan) data komentar pada sebuah video Youtube menggunakan **Youtube Data API v3**. Sebelum mencoba program ini, pastikan Anda sudah memiliki (mengaktifkan) layanan Youtube Data API dan telah membangkitkan **API Key**. 

Jika belum memiliki **API KEY**, Anda dapat mengikuti petunjuk singkat sebagai berikut: 
1. Login ke Google Developer Console (https://console.developers.google.com/)dengan akun Google Anda
2. Buat project baru dan lengkapi isian yang diminta. 
3. Aktifkan Layanan API pada halaman project, dan cari **Youtube Data API v3**.
4. Dari halaman dashboard, buat kredential agar API tersebut dapat digunakan. Klik tombol **Buat Kredensial** (**Create Credential**). Lengkapi isian formnya.
5. Anda dapat mengakses / melihat API KEY pada tab **Credentials**.



For more details : [link]

In [12]:
#install library
!pip install sastrawi
!pip install swifter
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
#import library
import pandas as pd
from googleapiclient.discovery import build
import numpy as np
from string import punctuation
import re
import nltk

### Load Data

In [14]:
#load data
df = pd.read_csv("https://raw.githubusercontent.com/triasmidwi/prosaindata/main/dataset/youtube_comment.csv", sep=';')
df.head()

,publishedAt,authorDisplayName,text,likeCount
0,2023-05-11T04:11:55Z,mujiati cantik,jaga wibawanya pak gp jgn cengar cengir,0
1,2023-05-11T04:05:47Z,Reza Rizal Ys,aku pilih rakyat biasa aja yaitu anies basweda...,0
2,2023-05-11T01:22:44Z,NINIEN S,hindari jauh penggemar video,0
3,2023-05-11T01:19:30Z,Nasrun SMD,bagi saya cuman ada calon ofsi yang bisa di pi...,0
4,2023-05-11T00:00:41Z,Zaen Arif,konten pkokkkkkk,0


### Preprocessing

#### 1. Symbol & Punctuation Removal, case folding

Pada Tahap ini preprocessing yang dilakukan yaitu menghilangkan simbol dan tanda baca, serta melakukan case folding yaitu merubah seluruh huruf yang ada pada data menjadi huruf kecil

In [15]:
#proses menghilangkan simbol dan emoji
def remove_text_special (text):
  text = text.replace('\\t',"").replace('\\n',"").replace('\\u',"").replace('\\',"")
  text = text.encode('ascii', 'replace').decode('ascii')
  return text.replace("http://"," ").replace("https://", " ")
df['text'] = df['text'].apply(remove_text_special)
print(df['text'])

0              jaga wibawanya  pak gp  jgn cengar cengir 
1       aku pilih rakyat biasa aja yaitu anies basweda...
2                            hindari jauh penggemar video
3       bagi saya cuman ada calon ofsi yang bisa di pi...
4                                        konten pkokkkkkk
                              ...                        
1701                                                bacot
1702                                          tepat skali
1703         dan apakan anda lebih mudeng  dan apakah ...
1704    begono bro  yen kowe arep ng semarang pilih ml...
1705    memangnya kalau seorang adalah petugas partai ...
Name: text, Length: 1706, dtype: object


In [16]:
#menghilangkan tanda baca
def remove_tanda_baca(text):
  text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text)
  return text

df['text'] = df['text'].apply(remove_tanda_baca)
df['text'].head(20)

0            jaga wibawanya  pak gp  jgn cengar cengir 
1     aku pilih rakyat biasa aja yaitu anies basweda...
2                          hindari jauh penggemar video
3     bagi saya cuman ada calon ofsi yang bisa di pi...
4                                      konten pkokkkkkk
5     wajah koruptor mnyesakan dada   kalo mbunuh di...
6     kasihan lambang islam dukung ci porno yg tiada...
7                                         setuju banget
8      pun oke  spy lebih fokus nantinya  br ingin p...
9                                       saya pilih saya
10                    salam damai semoga dpt yg trrbaik
11                                       klo lolos     
12     capres  capres yg bertarung cuma  capres anis...
13                        saya pilihan bangsa indonesia
14    untuk kebaikan bangsa yang kedepannya ada pote...
15    kami rakyat jawa tengah tidak akan mau pilih g...
16                                pak  anies  pilihanku
17                            ridwan kamil ada t

In [17]:
#proses menghilangkan angka
def remove_numbers (text):
  return re.sub(r"\d+", "", text)
df['text'] = df['text'].apply(remove_numbers)
df['text']

0              jaga wibawanya  pak gp  jgn cengar cengir 
1       aku pilih rakyat biasa aja yaitu anies basweda...
2                            hindari jauh penggemar video
3       bagi saya cuman ada calon ofsi yang bisa di pi...
4                                        konten pkokkkkkk
                              ...                        
1701                                                bacot
1702                                          tepat skali
1703         dan apakan anda lebih mudeng  dan apakah ...
1704    begono bro  yen kowe arep ng semarang pilih ml...
1705    memangnya kalau seorang adalah petugas partai ...
Name: text, Length: 1706, dtype: object

In [18]:
#proses casefolding
def casefolding(Comment):
  Comment = Comment.lower()
  return Comment
df['text'] = df['text'].apply(casefolding)
df['text']

0              jaga wibawanya  pak gp  jgn cengar cengir 
1       aku pilih rakyat biasa aja yaitu anies basweda...
2                            hindari jauh penggemar video
3       bagi saya cuman ada calon ofsi yang bisa di pi...
4                                        konten pkokkkkkk
                              ...                        
1701                                                bacot
1702                                          tepat skali
1703         dan apakan anda lebih mudeng  dan apakah ...
1704    begono bro  yen kowe arep ng semarang pilih ml...
1705    memangnya kalau seorang adalah petugas partai ...
Name: text, Length: 1706, dtype: object

#### 2. Tokenizing
Pada tahap ini preprocessing yang dilakukan adalah tokenizing. Tokenizing adalah metode untuk melakukan pemisahan kata dalam suatu kalimat dengan tujuan untuk proses analisis teks lebih lanjut

In [19]:
#proses tokenisasi
# from nltk.tokenize import TweetTokenizer
nltk.download('punkt')
# def word_tokenize(text):
#   tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
#   return tokenizer.tokenize(text)

df['review_token'] = df['text'].apply(lambda sentence: nltk.word_tokenize(sentence))
df['review_token']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0         [jaga, wibawanya, pak, gp, jgn, cengar, cengir]
1       [aku, pilih, rakyat, biasa, aja, yaitu, anies,...
2                       [hindari, jauh, penggemar, video]
3       [bagi, saya, cuman, ada, calon, ofsi, yang, bi...
4                                     [konten, pkokkkkkk]
                              ...                        
1701                                              [bacot]
1702                                       [tepat, skali]
1703    [dan, apakan, anda, lebih, mudeng, dan, apakah...
1704    [begono, bro, yen, kowe, arep, ng, semarang, p...
1705    [memangnya, kalau, seorang, adalah, petugas, p...
Name: review_token, Length: 1706, dtype: object

#### 3. Word Normalization
Pada tahap ini yang dilakukan yaitu normalisasi pada data. Hal tersebut dilakukan untuk merubah kata yang tidak baku menjadi kata baku

In [20]:
#Normalisasi kata tidak baku
normalize = pd.read_csv("https://raw.githubusercontent.com/triasmidwi/prosaindata/main/dataset/Normalization%20Data.csv")

normalize_word_dict = {}

for row in normalize.iterrows():
  if row[0] not in normalize_word_dict:
    normalize_word_dict[row[0]] = row[1]

def normalized_term(comment):
  return [normalize_word_dict[term] if term in normalize_word_dict else term for term in comment]

df['comment_normalize'] = df['review_token'].apply(normalized_term)
df['comment_normalize'].head(20)

0       [jaga, wibawanya, pak, gp, jgn, cengar, cengir]
1     [aku, pilih, rakyat, biasa, aja, yaitu, anies,...
2                     [hindari, jauh, penggemar, video]
3     [bagi, saya, cuman, ada, calon, ofsi, yang, bi...
4                                   [konten, pkokkkkkk]
5     [wajah, koruptor, mnyesakan, dada, kalo, mbunu...
6     [kasihan, lambang, islam, dukung, ci, porno, y...
7                                      [setuju, banget]
8     [pun, oke, spy, lebih, fokus, nantinya, br, in...
9                                   [saya, pilih, saya]
10             [salam, damai, semoga, dpt, yg, trrbaik]
11                                         [klo, lolos]
12    [capres, capres, yg, bertarung, cuma, capres, ...
13                   [saya, pilihan, bangsa, indonesia]
14    [untuk, kebaikan, bangsa, yang, kedepannya, ad...
15    [kami, rakyat, jawa, tengah, tidak, akan, mau,...
16                              [pak, anies, pilihanku]
17                      [ridwan, kamil, ada, tid

#### 4. Stopwords Removal
Pada tahap ini preprocessing yang dilakukan adalah menghilangkan kata yang tidak penting. Stopwords removal dilakukan 2 kali, yang pertama berdasarkan korpus yang ada di library python yaitu nltk, yang kedua berdasarkan file 'list_stopwords'

In [21]:
#Stopword Removal
nltk.download('stopwords')
from nltk.corpus import stopwords
txt_stopwords = stopwords.words('indonesian')

def stopwords_removal(filtering) :
  filtering = [word for word in filtering if word not in txt_stopwords]
  return filtering

df['stopwords_removal'] = df['comment_normalize'].apply(stopwords_removal)
df['stopwords_removal'].head(20)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0            [jaga, wibawanya, gp, jgn, cengar, cengir]
1     [pilih, rakyat, aja, anies, baswedan, yg, pro,...
2                           [hindari, penggemar, video]
3          [cuman, calon, ofsi, pilih, prabowo, ganjar]
4                                   [konten, pkokkkkkk]
5     [wajah, koruptor, mnyesakan, dada, kalo, mbunu...
6     [kasihan, lambang, islam, dukung, ci, porno, y...
7                                      [setuju, banget]
8          [oke, spy, fokus, br, perubahan, yg, hakiki]
9                                               [pilih]
10             [salam, damai, semoga, dpt, yg, trrbaik]
11                                         [klo, lolos]
12    [capres, capres, yg, bertarung, capres, anis, ...
13                         [pilihan, bangsa, indonesia]
14    [kebaikan, bangsa, kedepannya, potensi, perang...
15    [rakyat, jawa, pilih, ganjar, partai, pengusun...
16                                   [anies, pilihanku]
17                                  [ridwan, kam

In [22]:
#stopword removal 2
data_stopwords = pd.read_csv("https://raw.githubusercontent.com/triasmidwi/prosaindata/main/dataset/list_stopwords.csv")
print(data_stopwords)

def stopwords_removal2(filter) :
  filter = [word for word in filter if word not in data_stopwords]
  return filter

df['stopwords_removal_final'] = df['stopwords_removal'].apply(stopwords_removal2)
df['stopwords_removal_final'].head(20)

Empty DataFrame
Columns: [kah;samping;kiri;maju;mundur;orang kedua;youtube;nya;non;gaada;hehe;heheee;haha;bf;dll;un;safa;ida;idak;aja;kk;wkwkk;gimana;lihat;duel;boong;ya;coba;anjing;pilihan;sandi;seklai;anggi;tetima;ubtik;tai;t;f;a;v;lt;cq;fd;e;h;da;diba;ax;babu;tombol;kantuk;kntl;sherina;ter;eh;kitaudah;cari;daan;ketemu;sdang;luring;pas;luring;bilang;log;in;kembaliterimah;nge;lo;dakwahlbh;klan;detik;tolol;kaka;kakk;kakak;silah;mbaaak;mbak;kn;fasid;solo;loh;bagusbagus;an;nan;babi;negara;m;t;o;p;ii;nyacuma;putarpilihdengerinklo;sia;cv;katai;luring;log;tank;am;radio;simple;buk;bu;si;sih;daring;blibli;lainnya;keluarga;sim;mantaplahnice;luring;garena;garax;barca;yah;oh;heeh;rate;si;ini saya;resso;kaya;deh;pdkesmas;woi;kontol;loh;interface;gitu;mahasiswa;kobo;karnaeru;kenama;na;dulusetelah;berhasilsy;job;trm;ks;sangattttt;anak;ko;leon;hem;alhamdulillah;hai;chrome;book;thank;gandoss;lagulagunya;seharihari;laki;mah;aaaa;wkwk;iklanmau;mulumaksudnya;menh;maura;dafam;zakia;savita;full;dambil;yar

0            [jaga, wibawanya, gp, jgn, cengar, cengir]
1     [pilih, rakyat, aja, anies, baswedan, yg, pro,...
2                           [hindari, penggemar, video]
3          [cuman, calon, ofsi, pilih, prabowo, ganjar]
4                                   [konten, pkokkkkkk]
5     [wajah, koruptor, mnyesakan, dada, kalo, mbunu...
6     [kasihan, lambang, islam, dukung, ci, porno, y...
7                                      [setuju, banget]
8          [oke, spy, fokus, br, perubahan, yg, hakiki]
9                                               [pilih]
10             [salam, damai, semoga, dpt, yg, trrbaik]
11                                         [klo, lolos]
12    [capres, capres, yg, bertarung, capres, anis, ...
13                         [pilihan, bangsa, indonesia]
14    [kebaikan, bangsa, kedepannya, potensi, perang...
15    [rakyat, jawa, pilih, ganjar, partai, pengusun...
16                                   [anies, pilihanku]
17                                  [ridwan, kam

#### 5. Stemming
Pada tahap ini preprocessing yang dilakukan adalah stemming. Stemming adalah proses pemetaan dan penguraian bentuk dari suatu kata menjadi bentuk kata dasarnya.

In [23]:
#proses stem
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string
import swifter
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming (term):
  return stemmer.stem(term)

term_dict = {}

for document in df['stopwords_removal_final']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ''

for term in term_dict.keys():
  term_dict[term] = stemming(term)

def get_stemming(document):
  return [term_dict[term] for term in document]

In [24]:
df['stemming'] = df['stopwords_removal'].swifter.apply(get_stemming)

Pandas Apply:   0%|          | 0/1706 [00:00<?, ?it/s]

In [25]:
print(df['stemming'])

0                 [jaga, wibawa, gp, jgn, cengar, cengir]
1       [pilih, rakyat, aja, anies, baswedan, yg, pro,...
2                                  [hindar, gemar, video]
3            [cuman, calon, ofsi, pilih, prabowo, ganjar]
4                                     [konten, pkokkkkkk]
                              ...                        
1701                                              [bacot]
1702                                              [skali]
1703                    [apa, mudeng, tau, arti, politik]
1704    [begono, bro, yen, kowe, arep, ng, semarang, p...
1705    [memang, tugas, partai, nggak, gitu, tugas, ra...
Name: stemming, Length: 1706, dtype: object


In [26]:
df.head(20)

,publishedAt,authorDisplayName,text,likeCount,review_token,comment_normalize,stopwords_removal,stopwords_removal_final,stemming
0,2023-05-11T04:11:55Z,mujiati cantik,jaga wibawanya pak gp jgn cengar cengir,0,"[jaga, wibawanya, pak, gp, jgn, cengar, cengir]","[jaga, wibawanya, pak, gp, jgn, cengar, cengir]","[jaga, wibawanya, gp, jgn, cengar, cengir]","[jaga, wibawanya, gp, jgn, cengar, cengir]","[jaga, wibawa, gp, jgn, cengar, cengir]"
1,2023-05-11T04:05:47Z,Reza Rizal Ys,aku pilih rakyat biasa aja yaitu anies basweda...,0,"[aku, pilih, rakyat, biasa, aja, yaitu, anies,...","[aku, pilih, rakyat, biasa, aja, yaitu, anies,...","[pilih, rakyat, aja, anies, baswedan, yg, pro,...","[pilih, rakyat, aja, anies, baswedan, yg, pro,...","[pilih, rakyat, aja, anies, baswedan, yg, pro,..."
2,2023-05-11T01:22:44Z,NINIEN S,hindari jauh penggemar video,0,"[hindari, jauh, penggemar, video]","[hindari, jauh, penggemar, video]","[hindari, penggemar, video]","[hindari, penggemar, video]","[hindar, gemar, video]"
3,2023-05-11T01:19:30Z,Nasrun SMD,bagi saya cuman ada calon ofsi yang bisa di pi...,0,"[bagi, saya, cuman, ada, calon, ofsi, yang, bi...","[bagi, saya, cuman, ada, calon, ofsi, yang, bi...","[cuman, calon, ofsi, pilih, prabowo, ganjar]","[cuman, calon, ofsi, pilih, prabowo, ganjar]","[cuman, calon, ofsi, pilih, prabowo, ganjar]"
4,2023-05-11T00:00:41Z,Zaen Arif,konten pkokkkkkk,0,"[konten, pkokkkkkk]","[konten, pkokkkkkk]","[konten, pkokkkkkk]","[konten, pkokkkkkk]","[konten, pkokkkkkk]"
5,2023-05-10T23:32:22Z,Ama agus Kudrotilah,wajah koruptor mnyesakan dada kalo mbunuh di...,0,"[wajah, koruptor, mnyesakan, dada, kalo, mbunu...","[wajah, koruptor, mnyesakan, dada, kalo, mbunu...","[wajah, koruptor, mnyesakan, dada, kalo, mbunu...","[wajah, koruptor, mnyesakan, dada, kalo, mbunu...","[wajah, koruptor, mnyesakan, dada, kalo, mbunu..."
6,2023-05-10T23:28:28Z,Ama agus Kudrotilah,kasihan lambang islam dukung ci porno yg tiada...,0,"[kasihan, lambang, islam, dukung, ci, porno, y...","[kasihan, lambang, islam, dukung, ci, porno, y...","[kasihan, lambang, islam, dukung, ci, porno, y...","[kasihan, lambang, islam, dukung, ci, porno, y...","[kasihan, lambang, islam, dukung, ci, porno, y..."
7,2023-05-10T22:25:41Z,Bbg Susilo,setuju banget,0,"[setuju, banget]","[setuju, banget]","[setuju, banget]","[setuju, banget]","[tuju, banget]"
8,2023-05-10T20:05:26Z,Eko Waluyo,pun oke spy lebih fokus nantinya br ingin p...,0,"[pun, oke, spy, lebih, fokus, nantinya, br, in...","[pun, oke, spy, lebih, fokus, nantinya, br, in...","[oke, spy, fokus, br, perubahan, yg, hakiki]","[oke, spy, fokus, br, perubahan, yg, hakiki]","[oke, spy, fokus, br, ubah, yg, hakiki]"
9,2023-05-10T15:34:31Z,Prof. Dr. dr. H SUGIONO,saya pilih saya,0,"[saya, pilih, saya]","[saya, pilih, saya]",[pilih],[pilih],[pilih]


## Feature Extraction (TF-IDF)

Algoritma TF-IDF (Term Frequency – Inverse Document Frequency) adalah salah satu algoritma yang dapat digunakan untuk menganalisa hubungan antara sebuah frase/kalimat dengan sekumpulan dokumen. Contoh yang dibahas kali ini adalah mengenai penentuan urutan peringkat data berdasarkan query yang digunakan.
Inti utama dari algoritma ini adalah melakukan perhitungan nilai TF dan nilai IDF dari sebuah setiap kata kunci terhadap masing-masing dokumen

In [27]:
def joinkata(data):
  kalimat = ""
  for i in data:
    kalimat += i
    kalimat += " "
  return kalimat

text = df['stemming'].swifter.apply(joinkata)
text

Pandas Apply:   0%|          | 0/1706 [00:00<?, ?it/s]

0                       jaga wibawa gp jgn cengar cengir 
1       pilih rakyat aja anies baswedan yg pro rakyat ...
2                                     hindar gemar video 
3                  cuman calon ofsi pilih prabowo ganjar 
4                                       konten pkokkkkkk 
                              ...                        
1701                                               bacot 
1702                                               skali 
1703                         apa mudeng tau arti politik 
1704    begono bro yen kowe arep ng semarang pilih mla...
1705    memang tugas partai nggak gitu tugas rakyat me...
Name: stemming, Length: 1706, dtype: object

In [28]:
# Vectorize document using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words='english',
                        ngram_range = (1,1)
                        )

# Fit and Transform the documents
tfidf_matrix = vectorizer.fit_transform(text)

In [29]:
print(tfidf_matrix)

  (0, 568)	0.4479029703993446
  (0, 566)	0.4479029703993446
  (0, 1248)	0.2775453269716463
  (0, 964)	0.3686597537918155
  (0, 3338)	0.4479029703993446
  (0, 1202)	0.4303572524457923
  (1, 2286)	0.23610243826107458
  (1, 3219)	0.29239947287836926
  (1, 2511)	0.48667747088833074
  (1, 3400)	0.19136178150038505
  (1, 312)	0.34465432990624456
  (1, 149)	0.26399122201876446
  (1, 61)	0.29239947287836926
  (1, 2572)	0.5187090843582427
  (1, 2393)	0.20554648247286164
  (2, 3298)	0.53779343758486
  (2, 916)	0.607933580317421
  (2, 1079)	0.5841189779600566
  (3, 890)	0.2509473885689311
  (3, 2478)	0.25411437946938503
  (3, 2202)	0.6639247358399525
  (3, 532)	0.3489578861670763
  (3, 627)	0.5104158458552951
  (3, 2393)	0.22216951509782296
  (4, 2418)	0.725840665873398
  :	:
  (1704, 1953)	0.4000013912208075
  (1704, 2056)	0.20815474849495794
  (1704, 478)	0.1740293758765993
  (1704, 1985)	0.20000069561040376
  (1704, 549)	0.15168136176910277
  (1704, 2286)	0.08442708452996386
  (1704, 2393)	0.0

### Latent Semantic Analysis

In [30]:
# Melakukan dekomposisi matriks dengan SVD
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=4)
lsa_matrix = svd_model.fit_transform(tfidf_matrix)

## Modelling

Bobot kata terhadap masing masing topik

In [31]:
# bobot kata terhadap masing masing topik
terms = vectorizer.get_feature_names_out()

for index, component in enumerate(svd_model.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:2]
    print("Topic "+str(index)+": ",top_terms_key)

Topic 0:  [('prabowo', 0.4610633584741466), ('pilih', 0.41489318252069685)]
Topic 1:  [('prabowo', 0.827803783722586), ('mahfud', 0.06375979070535973)]
Topic 2:  [('ganjar', 0.599322589465989), ('anies', 0.3344833759586806)]
Topic 3:  [('ganjar', 0.6379378319143381), ('pranowo', 0.13258527398675007)]


Bobot setiap topik terhadap dokumen

In [32]:
# bobot setiap topik terhadap  dokumen
df_lsa = pd.DataFrame(lsa_matrix, columns=["Topik 0", "Topik 1", "Topik 2", "Topik 3"])
df_lsa = pd.concat([text, df_lsa], axis=1)
df_lsa['Topik']= df_lsa[['Topik 0', 'Topik 1', 'Topik 2', 'Topik 3']].apply(lambda x: x.argmax(), axis=1)

df_lsa

,stemming,Topik 0,Topik 1,Topik 2,Topik 3,Topik
0,jaga wibawa gp jgn cengar cengir,1.305339e-02,-6.599097e-03,1.760224e-03,-1.948879e-03,0
1,pilih rakyat aja anies baswedan yg pro rakyat ...,4.710381e-01,-3.385620e-01,-9.703459e-02,-1.605835e-01,0
2,hindar gemar video,2.848306e-03,-2.527921e-03,-1.085127e-03,2.007129e-03,0
3,cuman calon ofsi pilih prabowo ganjar,3.093808e-01,1.734764e-01,1.053735e-01,1.505812e-01,0
4,konten pkokkkkkk,1.079018e-03,-4.920639e-04,1.650286e-03,1.462248e-03,2
...,...,...,...,...,...,...
1701,bacot,-8.490512e-09,-3.377479e-08,1.853962e-07,-9.170537e-08,2
1702,skali,2.402701e-03,5.991124e-03,-1.952073e-03,-7.432785e-04,1
1703,apa mudeng tau arti politik,1.667986e-02,-1.157020e-02,-2.040573e-03,5.584301e-03,0
1704,begono bro yen kowe arep ng semarang pilih mla...,5.953209e-02,-4.118933e-02,-3.344795e-02,6.632529e-03,0
